In [ ]:
# Core
import numpy as np
import pandas as pd

# SciPy / sklearn
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import (
    VarianceThreshold, SelectKBest, SelectFdr, f_classif, r_regression
)
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score, recall_score, roc_curve
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier

# Optional: imbalanced-learn & mrmr
try:
    from imblearn.ensemble import EasyEnsembleClassifier
    from imblearn.over_sampling import SMOTE
except ImportError:
    EasyEnsembleClassifier = None
    SMOTE = None

try:
    import mrmr  # if not available, we fallback later
except ImportError:
    mrmr = None

# Utilities
from pathlib import Path
from typing import Tuple, List, Optional, Dict
from joblib import dump
from datetime import datetime

RANDOM_STATE = 42

In [ ]:
def read_csv(file_root: str, hk: bool = False) -> Tuple[pd.DataFrame, pd.Series, pd.Series]:
    """Read CSV, process 'event', and return (features, label, patient_id)."""
    data = pd.read_csv(file_root)
    data = data.dropna(subset=['event'])
    if hk:
        data['event'] = data['event'].replace({'TRUE': True, '0': False})
    if 'time' not in data.columns:
        raise ValueError("'time' column is required for downstream analysis.")
    patient = data['ID']
    label = data['event']
    # Feature block (adjust if your features live elsewhere)
    train_data = data.iloc[:, 1:754].copy()
    for col in ['AGE', 'SEX', 'BMI']:
        if col in data.columns:
            train_data[col] = data[col]
    return train_data, label, patient


def confounding_correlation_reduction(feature_table: pd.DataFrame,
                                      confounding_factor_table: pd.DataFrame) -> pd.DataFrame:
    """Remove features highly associated with confounding factors."""
    common_index = feature_table.dropna().index.intersection(confounding_factor_table.dropna().index)
    for confounding_factor in confounding_factor_table.columns:
        unique_values = confounding_factor_table[confounding_factor].unique()
        if len(unique_values) < 5:
            _, p_values = f_classif(
                feature_table.loc[common_index].values,
                confounding_factor_table.loc[common_index, confounding_factor].values
            )
            mask = p_values > 0.05
        else:
            r = r_regression(
                feature_table.loc[common_index].values,
                confounding_factor_table.loc[common_index, confounding_factor].values
            )
            mask = np.abs(r) < 0.65
        feature_table = feature_table.loc[:, mask]
    return feature_table


def select_feature(train_data_collected: pd.DataFrame, para_factors: Dict) -> Tuple[pd.DataFrame, List[str]]:
    """Keyword exclusion + confounder filtering."""
    excluding_keywords = para_factors.get('Excluding keywords')
    if excluding_keywords:
        selected_features = [
            name for name in train_data_collected.columns
            if not any(keyword in name for keyword in excluding_keywords)
        ]
        selected_training_features = train_data_collected[selected_features].copy()
        print(f"{selected_training_features.shape[1]} features remain after keyword exclusion.")
    else:
        selected_training_features = train_data_collected.copy()

    confounding_factors = para_factors.get('Confounding factors')
    if confounding_factors:
        confounding_tables = [
            train_data_collected[c] for c in confounding_factors if c in train_data_collected.columns
        ]
        if confounding_tables:
            confounding_df = pd.concat(confounding_tables, axis=1)
            selected_training_features = confounding_correlation_reduction(selected_training_features, confounding_df)
            print(f"{selected_training_features.shape[1]} features remain after confounder filtering.")
    return selected_training_features, list(selected_training_features.columns)


def extract_pd_values(data_df: pd.DataFrame, label_pf: pd.Series) -> Tuple[np.ndarray, np.ndarray]:
    return data_df.values, label_pf.astype(int).values


def classification_model_performance(model, predictors, ground_truth, return_roc=False):
    probs = model.predict_proba(predictors)[:, 1]
    preds = model.predict(predictors)
    scores = {
        'AUC': roc_auc_score(ground_truth, probs),
        'Accuracy': accuracy_score(ground_truth, preds),
        'Precision': precision_score(ground_truth, preds),
        'Recall': recall_score(ground_truth, preds),
    }
    scores = pd.Series(scores)
    if return_roc:
        fpr, tpr, thresholds = roc_curve(ground_truth, probs)
        roc_df = pd.DataFrame({'FPR': fpr, 'TPR': tpr, 'Threshold': thresholds})
        return scores, roc_df
    return scores

In [ ]:
def mrmr_names_or_fallback(X_df: pd.DataFrame, y: pd.Series, k: int) -> List[str]:
    """Return top-k feature names via mRMR; fallback to SelectKBest(f_classif) if not installed."""
    if mrmr is not None:
        return mrmr.mrmr_classif(X=X_df, y=y, K=k)
    print("[WARN] 'mrmr' not installed; falling back to SelectKBest(f_classif)."
          " Results may differ from manuscript.")
    skb = SelectKBest(f_classif, k=min(k, X_df.shape[1]))
    skb.fit(X_df.values, y.values)
    return list(X_df.columns[skb.get_support()])


def train_model(classifier: str, train_data, train_label):
    classifier = classifier.upper()
    model_map = {
        'LR': LogisticRegression(random_state=RANDOM_STATE, max_iter=1000),
        'SVC': SVC(probability=True, random_state=RANDOM_STATE),
        'RF': RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE),
        'GB': GradientBoostingClassifier(random_state=RANDOM_STATE),
        'NB': GaussianNB(),
    }
    if classifier not in model_map:
        raise ValueError(f"Unknown classifier '{classifier}'. Choose from LR, SVC, RF, GB, NB.")
    model = model_map[classifier]
    model.fit(train_data, np.ravel(train_label))
    return model


def evaluate_model(model, test_data, test_label):
    probs = model.predict_proba(test_data)[:, 1]
    return roc_auc_score(np.ravel(test_label), probs)


def print_results(mode_str, train_auc, val_auc, test_auc, evaluation_type):
    print(f"\n[{evaluation_type}] Mode: {mode_str}")
    print(f"Training AUC:   {train_auc:.4f}" if train_auc is not None else "Training AUC:   N/A")
    print(f"Validation AUC: {val_auc:.4f}"   if val_auc is not None else "Validation AUC: N/A")
    print(f"Test AUC:       {test_auc:.4f}\n" if test_auc is not None else "Test AUC:       N/A")


def save_models(models: dict, out_dir: str | Path = "artifacts/models", prefix: str | None = None) -> dict:
    """Save trained sklearn models (.joblib)."""
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    tag = prefix if prefix is not None else datetime.now().strftime("%Y%m%d-%H%M%S")
    saved = {}
    for name, model in models.items():
        if model is None:
            continue
        fname = f"{tag}_{name}.joblib"
        fpath = out_dir / fname
        dump(model, fpath)
        saved[name] = str(fpath)
        print(f"[saved] {name}: {fpath}")
    return saved

In [ ]:
# --- Config & constants -------------------------------------------------------
DATA_ROOT = Path("data")

PATHS = {
    'us_train': DATA_ROOT / 'us_train.csv',
    'us_val':   DATA_ROOT / 'us_val.csv',
    'us_test':  DATA_ROOT / 'us_test.csv',
    'hk_train': DATA_ROOT / 'hk_train.csv',
    'hk_val':   DATA_ROOT / 'hk_val.csv',
    'hk_test':  DATA_ROOT / 'hk_test.csv',
    'uk_test':  DATA_ROOT / 'uk_test.csv',   # optional external test
}

para_factors = {
    'Confounding factors': ['original_shape2D_PixelSurface'],
    'Excluding keywords': ['shape'],
    'MRMR feature number': 24,
}

N_mrmr_ft = para_factors['MRMR feature number']

In [ ]:
# Load US dataset (already merged) and apply filtering + mRMR
us_train_data, us_train_label, us_train_patient = read_csv(str(PATHS['us_train']))
us_val_data,   us_val_label,   us_val_patient   = read_csv(str(PATHS['us_val']))
us_test_data,  us_test_label,  us_test_patient  = read_csv(str(PATHS['us_test']))

us_train_data_hat, _ = select_feature(us_train_data, para_factors)
us_mrmr_names = mrmr_names_or_fallback(us_train_data_hat, us_train_label, N_mrmr_ft)

# Load HK dataset and apply filtering + mRMR
hk_train_data, hk_train_label, hk_train_patient = read_csv(str(PATHS['hk_train']), hk=True)
hk_val_data,   hk_val_label,   hk_val_patient   = read_csv(str(PATHS['hk_val']),   hk=True)
hk_test_data,  hk_test_label,  hk_test_patient  = read_csv(str(PATHS['hk_test']),  hk=True)

hk_train_data_hat, _ = select_feature(hk_train_data, para_factors)
hk_mrmr_names = mrmr_names_or_fallback(hk_train_data_hat, hk_train_label, N_mrmr_ft)

# Intersection as robust shared features
shared_feat_names = list(set(us_mrmr_names).intersection(hk_mrmr_names))
print("Number of intersected features selected by mRMR:", len(shared_feat_names))
print("US top (mRMR):", us_mrmr_names)
print("HK top (mRMR):", hk_mrmr_names)

In [ ]:
def _to_np(Xdf, yser):
    return extract_pd_values(Xdf, yser)

# US own mRMR set
us_train_X = us_train_data[us_mrmr_names];  us_train_X, us_train_y = _to_np(us_train_X, us_train_label)
us_val_X   = us_val_data[us_mrmr_names];    us_val_X,   us_val_y   = _to_np(us_val_X,   us_val_label)
us_test_X  = us_test_data[us_mrmr_names];   us_test_X,  us_test_y  = _to_np(us_test_X,  us_test_label)

# HK own mRMR set
hk_train_X = hk_train_data[hk_mrmr_names];  hk_train_X, hk_train_y = _to_np(hk_train_X, hk_train_label)
hk_val_X   = hk_val_data[hk_mrmr_names];    hk_val_X,   hk_val_y   = _to_np(hk_val_X,   hk_val_label)
hk_test_X  = hk_test_data[hk_mrmr_names];   hk_test_X,  hk_test_y  = _to_np(hk_test_X,  hk_test_label)

# Shared feature view (for combined model)
us_shared_train_X = us_train_data[shared_feat_names]; us_shared_train_X, us_shared_train_y = _to_np(us_shared_train_X, us_train_label)
us_shared_val_X   = us_val_data[shared_feat_names];   us_shared_val_X,   us_shared_val_y   = _to_np(us_shared_val_X,   us_val_label)
us_shared_test_X  = us_test_data[shared_feat_names];  us_shared_test_X,  us_shared_test_y  = _to_np(us_shared_test_X,  us_test_label)

hk_shared_train_X = hk_train_data[shared_feat_names]; hk_shared_train_X, hk_shared_train_y = _to_np(hk_shared_train_X, hk_train_label)
hk_shared_val_X   = hk_val_data[shared_feat_names];   hk_shared_val_X,   hk_shared_val_y   = _to_np(hk_shared_val_X,   hk_val_label)
hk_shared_test_X  = hk_test_data[shared_feat_names];  hk_shared_test_X,  hk_shared_test_y  = _to_np(hk_shared_test_X,  hk_test_label)

In [ ]:
classifier = 'GB'  # choose from: LR, SVC, RF, GB, NB

# Train within-domain models (own mRMR sets)
model_us = train_model(classifier, us_train_X, us_train_y)
train_auc_us = evaluate_model(model_us, us_train_X, us_train_y)
val_auc_us   = evaluate_model(model_us, us_val_X,   us_val_y)
test_auc_us  = evaluate_model(model_us, us_test_X,  us_test_y)
print_results("us -> us", train_auc_us, val_auc_us, test_auc_us, "Initial Evaluation")

model_hk = train_model(classifier, hk_train_X, hk_train_y)
train_auc_hk = evaluate_model(model_hk, hk_train_X, hk_train_y)
val_auc_hk   = evaluate_model(model_hk, hk_val_X,   hk_val_y)
test_auc_hk  = evaluate_model(model_hk, hk_test_X,  hk_test_y)
print_results("hk -> hk", train_auc_hk, val_auc_hk, test_auc_hk, "Initial Evaluation")

# Cross-dataset (own mRMR sets)
test_auc_us_hk = evaluate_model(model_us, hk_test_X, hk_test_y)
print_results("us -> hk", train_auc_us, val_auc_us, test_auc_us_hk, "Cross-Dataset Evaluation")

test_auc_hk_us = evaluate_model(model_hk, us_test_X, us_test_y)
print_results("hk -> us", train_auc_hk, val_auc_hk, test_auc_hk_us, "Cross-Dataset Evaluation")

# Combined/shared-features model trained on US shared features, then tested on each domain's shared test
model_combined = train_model(classifier, us_shared_train_X, us_shared_train_y)
test_auc_combined_us = evaluate_model(model_combined, us_shared_test_X, us_shared_test_y)
print_results("Combined (shared) -> us", None, None, test_auc_combined_us, "Expected Outcome (After Proposed Solution)")

test_auc_combined_hk = evaluate_model(model_combined, hk_shared_test_X, hk_shared_test_y)
print_results("Combined (shared) -> hk", None, None, test_auc_combined_hk, "Expected Outcome (After Proposed Solution)")

In [ ]:
# Optional: UK external test set (if present)
if PATHS['uk_test'].exists():
    uk_test_data, uk_test_label, uk_test_patient = read_csv(str(PATHS['uk_test']), hk=True)

    # Shared features with UK
    uk_shared_test_X = uk_test_data[shared_feat_names]; uk_shared_test_X, uk_shared_test_y = extract_pd_values(uk_shared_test_X, uk_test_label)

    # US/HK own mRMR projections for UK (only if columns exist); guard with set intersection
    us_uk_cols = [c for c in us_mrmr_names if c in uk_test_data.columns]
    hk_uk_cols = [c for c in hk_mrmr_names if c in uk_test_data.columns]

    if us_uk_cols:
        ukus_test_X = uk_test_data[us_uk_cols]; ukus_test_X, ukus_test_y = extract_pd_values(ukus_test_X, uk_test_label)
        test_auc_ukus = evaluate_model(model_us, ukus_test_X, ukus_test_y)
        print_results("us -> uk", None, None, test_auc_ukus, "Initial Evaluation")
    else:
        print("[INFO] No overlapping columns for US mRMR set in UK test; skipping us->uk.")

    if hk_uk_cols:
        ukhk_test_X = uk_test_data[hk_uk_cols]; ukhk_test_X, ukhk_test_y = extract_pd_values(ukhk_test_X, uk_test_label)
        test_auc_ukhk = evaluate_model(model_hk, ukhk_test_X, ukhk_test_y)
        print_results("hk -> uk", None, None, test_auc_ukhk, "Initial Evaluation")
    else:
        print("[INFO] No overlapping columns for HK mRMR set in UK test; skipping hk->uk.")

    test_auc_combined_uk = evaluate_model(model_combined, uk_shared_test_X, uk_shared_test_y)
    print_results("Combined (shared) -> uk", None, None, test_auc_combined_uk, "Expected Outcome (After Proposed Solution)")
else:
    print("[INFO] PATHS['uk_test'] not found; skipping UK external evaluation.")

In [ ]:
# Save trained models (joblib). Change keys as you prefer.
saved_paths = save_models(
    models={
        'us': model_us,
        'hk': model_hk,
        'combined': model_combined,
    },
    out_dir='models'
)
saved_paths